In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# constants
#path = 'C:/Users/monte/Documents/Memoria/src'
path = 'C:/Users/asus/Documents/2023_2/Memoria/src'

# Ostar
O_df = pd.read_csv(path+'/Data/o_spectral_lines.csv')
# Bstar
B_df = pd.read_csv(path+'/Data/b_spectral_lines.csv')
# Observed spectra
Obs_df = pd.read_csv(path+'/Data/observed_spectral_lines.csv')

In [2]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers 

#input_shape = (4, 10, 128) = (batch_size, steps, vectorLength) = (batchsize, rotaciones, frecuencias)

#normalize = layers.Normalization()

# This layer will shift and scale inputs into a distribution centered around 0 with standard deviation 1.
# It accomplishes this by precomputing the mean and variance of the data and calling (input - mean) / sqrt(var) at runtime

# TODO: Probar con los datos sin normalizar
#normalize.adapt(x_train)

#input_shape = (x_train.shape[1],1)

def generate_model(): 
  return tf.keras.Sequential([
  keras.Input(shape=(701,1)),
  #normalize,
  layers.Conv1D(32,padding="same",kernel_size=3, activation='relu'),
  layers.Conv1D(32,padding="same",kernel_size=3, activation='relu'),
  layers.Conv1D(32,padding="same",kernel_size=3, activation='relu'),
  layers.MaxPooling1D(pool_size=3, padding='same'),
  layers.Conv1D(64,padding="same",kernel_size=2, activation='relu'),
  layers.Conv1D(64,padding="same",kernel_size=2, activation='relu'),
  layers.Conv1D(64,padding="same",kernel_size=2, activation='relu'),
  layers.MaxPooling1D(pool_size=2, padding='same'),
  layers.Conv1D(128,padding="same",kernel_size=2, activation='relu'),
  layers.Conv1D(128,padding="same",kernel_size=2, activation='relu'),
  layers.Conv1D(128,padding="same",kernel_size=2, activation='relu'),
  layers.MaxPooling1D(pool_size=2, padding='same'),
  layers.GlobalAveragePooling1D(),
  
  layers.Dense(128, activation='relu'),
  layers.Dense(3, activation='linear')
])

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

target = ['temperature','g','v']


X = B_df.drop(target,axis = 1)
Y = B_df[target]

X_obs = Obs_df.drop(target,axis=1)
Y_obs = Obs_df[target]

y_scaler = MinMaxScaler()

x_train, x_val, y_train, y_val = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=42)
x_obs = X_obs

#x_train = np.random.normal(loc=0, scale=0.01, size=x_train.shape) + x_train

y_scaler = y_scaler.fit(y_train)
y_train = y_scaler.transform(y_train)
y_val = y_scaler.transform(y_val)
y_obs = y_scaler.transform(Y_obs)


In [6]:
from keras.optimizers import Adam

model = generate_model()

model.compile(loss='MSE',
		      optimizer=Adam(),
		      metrics=['MAE'])

history = model.fit(
  	x = x_train,
  	y = y_train,
		epochs=10,
		validation_data=(x_val,y_val))


Epoch 1/10
612/612 [==============================] - 21s 19ms/step - loss: 0.0857 - MAE: 0.2498 - val_loss: 0.0847 - val_MAE: 0.2495
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0826 - MAE: 0.2466 - val_loss: 0.0823 - val_MAE: 0.2472
Epoch 3/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0822 - MAE: 0.2462 - val_loss: 0.0821 - val_MAE: 0.2465
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0821 - MAE: 0.2460 - val_loss: 0.0825 - val_MAE: 0.2469
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0821 - MAE: 0.2461 - val_loss: 0.0820 - val_MAE: 0.2463
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0821 - MAE: 0.2461 - val_loss: 0.0830 - val_MAE: 0.2474
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0821 - MAE: 0.2461 - val_loss: 0.0820 - val_MAE: 0.2464
Epoch 8/10
612/612 [==============================] - 11s 18ms

In [8]:
predictions = y_scaler.inverse_transform(model.predict(x_obs))

1/1 [==============================] - 0s 40ms/step


In [9]:
predictions

array([[2.1994469e+04, 3.5939684e+00, 2.3678839e+02],
       [2.1994326e+04, 3.5940027e+00, 2.3679062e+02],
       [2.1994346e+04, 3.5939505e+00, 2.3678801e+02],
       [2.1994209e+04, 3.5939736e+00, 2.3678862e+02],
       [2.1994209e+04, 3.5939736e+00, 2.3678862e+02],
       [2.1994090e+04, 3.5939119e+00, 2.3678001e+02],
       [2.1994469e+04, 3.5939684e+00, 2.3678839e+02],
       [2.1994209e+04, 3.5939736e+00, 2.3678862e+02],
       [2.1994350e+04, 3.5939393e+00, 2.3678642e+02]], dtype=float32)

In [12]:
Y_obs

,temperature,g,v
0,15000.0,3.40,68.0
1,18400.0,3.50,40.0
2,30000.0,3.53,150.0
3,22000.0,3.90,280.0
4,24500.0,3.40,215.0
5,22000.0,3.50,180.0
6,20000.0,3.56,10.0
7,18000.0,3.53,250.0
8,26000.0,2.90,300.0
